In [4]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt


In [5]:
BASE_DIR = "PlantVillage"
IMG_SIZE = 256 # check image proprties
BATCH_SIZE = 32 # standard convention 
EPOCH = 50 # at end of every epoch, we do validation
dataset = tf.keras.preprocessing.image_dataset_from_directory(
        directory=BASE_DIR, 
        shuffle=True, 
        image_size=(IMG_SIZE, IMG_SIZE), 
        batch_size=BATCH_SIZE
    )


Found 2152 files belonging to 3 classes.


In [8]:
class_names = dataset.class_names
class_names

['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

In [9]:
'''
Total images = 2152
Batch size = 32 
Total batches = 2152/32 = 68 = length of dataset
80% training data = 54 batches
10% validation data = 6 batches
10% testing data = 8 batches
'''
train_ds = dataset.take(54) #80% data is training
len(train_ds)

54

In [10]:
non_train_ds = dataset.skip(54)
val_ds = non_train_ds.take(6)
len(val_ds)

6

In [11]:
test_ds = non_train_ds.skip(6)
len(test_ds)

8